Optimisation des controles pour une zone et un pas de temps en tenant compte du confort, energie, pique de puissance et l'équipement.

### Variables: 

u<sub>i</sub> , i = -2, -2, 0, 1, 2. Représente chaque action.

Les u<sub>i</sub> sont binaires et = 1 si la controle i est allumée et = 0 sinon.

### Objectif:

Min $\alpha_1$ $f_{energy}$ + $\alpha_2$ $f_{comfort}$ + $\alpha_3$ $f_{power}$ + $\alpha_4$ $f_{equipment}$

$f_{energy}$ = $P$

$f_{comfort}$ = | $T$ + $\Delta$ $T$ - $T_{set}$ |

$f_{power}$ = $max(0, P - P_{limit})$

$f_{equipment}$ = | control - previous_control |

Énergie: Minimiser la puissance à un pas de temps.

Confort: Minimiser la distance entre la température au prochain pas et une température optimal prédéfinie.

Puissance: On définit une limite de  puissance et on minimise la distance entre la puissance et cette limite si la puissance est au dela de la limite. Sinon, cet objectif n'entre pas en compte.

Équipement: On minimise la distance entre la controle et la controle précédente. Par exemple, changer de 0 à 1 (f=1) est mieux que de changer de -2 à 2 (f=4).

En variant les pondérations, on peut trouver un compromis entre les différents aspects.

### Contraintes:

(1) $\sum_{i=-2}^{2} u_i$ = 1

On veut seulement une action.

(2) $T_{min} \leq T_{delta} \leq T_{max}$

Un interval de température assez large assure que la température reste dans des bornes acceptables.

In [2]:
using JuMP
using HiGHS
using JSON

function get_model(settings)
    
    model = Model(HiGHS.Optimizer)

    power = settings["power"]
    delta = settings["delta"]
    controls = settings["controls"]
    T = settings["T_init"]
    T_min = settings["T_min"]
    T_max = settings["T_max"]
    T_set = settings["T_set"]
    energy = settings["energy"]
    p_limit = settings["power_limit"]
    previous_control = settings["previous_control"]
    comfort_weight = settings["comfort_weight"]
    energy_weight = settings["energy_weight"]
    power_weight = settings["power_weight"]
    equipment_weight = settings["equipment_weight"]

    @variable(
        model,
        control[i in -2:2],
        Bin
    )

    comfort_coeffs = map(t -> abs(T + t - T_set), delta)
    energy_coeffs = power
    power_coeffs = map(t -> maximum([0, t - p_limit]), power)
    equipment_coeffs = map(t -> abs(t - previous_control), controls)

    
    @objective(
        model,
        Min,
        comfort_weight .* (sum(control .* comfort_coeffs))
        + energy_weight .* (sum(control .* energy_coeffs))
        + power_weight .* (sum(control .* power_coeffs))
        + equipment_weight .* (sum(control .* equipment_coeffs))
    )

    delta_T = sum(control .* delta)
    @constraint(
        model,
        temp_bounds_cons,
        T_min <= T + delta_T <= T_max
    )

    @constraint(
        model,
        garantee_action_cons,
        sum(control) == 1
    )

    return model

end

get_model (generic function with 1 method)

In [16]:
function main()

    settings = JSON.parse("""
    {
        "power": [2,1,0,1,2],
        "delta": [-2, -1, 1, 2, 3],
        "controls": [-2, -1, 0, 1, 2],
        "T_init": 20,
        "T_min": 17,
        "T_max": 24,
        "T_set": 20,
        "energy": 0,
        "power_limit": 1,
        "previous_control": 0,
        "comfort_weight": 1,
        "energy_weight": 1,
        "power_weight": 1,
        "equipment_weight": 1
    }""")
    model = get_model(settings)

    optimize!(model)
    solution_summary(model, verbose=true)
    
end

main()

Presolving model
2 rows, 5 cols, 10 nonzeros
2 rows, 5 cols, 9 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   5 cols (5 binary, 0 integer, 0 implied int., 0 continuous)
   9 nonzeros

( 0.0s) Starting symmetry detection
( 0.0s) No symmetry present

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      1
  Dual bound        1
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    1 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
         

* Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 1.00000e+00
  Objective bound      : 1.00000e+00
  Relative gap         : 0.00000e+00
  Primal solution :
    control[-1] : 0.00000e+00
    control[-2] : 0.00000e+00
    control[0] : 1.00000e+00
    control[1] : 0.00000e+00
    control[2] : 0.00000e+00

* Work counters
  Solve time (sec)   : 9.96113e-04
  Simplex iterations : -1
  Barrier iterations : -1
  Node count         : 1
